# Introduction

# Imports

In [ ]:
from pathlib import Path
from harbor.plotting.ligands import plot_aligned_ligands

In order to avoiding having this repo depend directly on the asapdiscovery repo, I'm going to comment this out, but we'll use a test example from the covid moonshot molecules:
```
from asapdiscovery.data.testing.test_resources import fetch_test_file
mypath = fetch_test_file("Mpro_combined_labeled.sdf")
``` 

In [ ]:
mypath = Path("../data/Mpro_combined_labeled.sdf")

I'm copying this code from the asapdiscovery repo.
Once that is conda installable, I'll make that a dep of this repo and use those tools for loading molecules

# Load Molecules

In [ ]:
from rdkit import Chem
mols = Chem.SDMolSupplier(mypath)

In [ ]:
mols = [mol for mol in mols]

In [ ]:
import mols2grid

In [ ]:
# define the grid to show the scafffolds
grid = mols2grid.display(mols)

In [ ]:
grid